# Ridge regression

In [1]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import Ridge 

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target

In [2]:
def fit_inverse(X, y, alpha=1.0, fit_intercept=False):
    """Direct method using the inverse"""
    # YOUR CODE HERE
    
    # With Bias
    
    # Identity matrix
    I = np.eye(X.shape[1])
    
    if fit_intercept == True:
            temp = np.ones((X.shape[0], 1))
            X = np.concatenate((temp, X), axis=1)
            I = np.eye(X.shape[1])
            I[0][0] = 0
            
    return scipy.linalg.inv(X.T @ X + alpha * I) @ (X.T @ y)

In [3]:
w = fit_inverse(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
assert error <= 1e-11
print("prediction error: ", error)

prediction error:  8.322327155333951e-15


In [4]:
def fit_qr_linear_regression(X, y):
    """ QR approach for linear regression from previous assignement"""
    # YOUR CODE HERE
    
    # Using (XT.X) = q.r
    q , r = np.linalg.qr(X.T @ X)
    
    # q.r.w = XT.y
    y1 = X.T @ y
    
    # z1 = r.w and we solve q.z1 = y1
    z1 = np.linalg.solve(q , y1)
    
    # We solve r.w = z1 
    return np.linalg.solve(r , z1)

def fit_qr(X, y, alpha=1.0, fit_intercept=False):
    """QR approach"""
    
    # Identity matrix
    I = np.eye(X.shape[1])
    
    # with bias
    if fit_intercept == True:
            temp = np.ones((X.shape[0], 1))
            X = np.concatenate((temp, X), axis=1)
            I = np.eye(X.shape[1])
            I[0][0] = 0
            
    X1 = np.sqrt(alpha * I)
    y1 = np.zeros(X.shape[1])
    
    X = np.vstack((X , X1))
    y = np.hstack((y , y1))
    
    return fit_qr_linear_regression(X, y)
    

In [5]:
w = fit_qr(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
#assert error <= 1e-11
print("prediction error: ", error)

prediction error:  1.0173620338866472e-13


In [6]:
def fit_svd(X, y, alpha=1.0, fit_intercept=False):
    """SVD approach"""
    # YOUR CODE HERE
    
    # Identity matrix
    I = np.eye(X.shape[1])
    
    # with bias
    if fit_intercept == True:
            temp = np.ones((X.shape[0], 1))
            X = np.concatenate((temp, X), axis=1)
            I = np.eye(X.shape[1])
            I[0][0] = 0
    
    # Using (XT.X) = U.S.VT
    U , S , Vt = np.linalg.svd(X)
    
    m = (X.shape[0]-X.shape[1],X.shape[1])
    
    S = np.vstack((np.diag(S),np.zeros(m)))
    A = S.T @ S + alpha * I
    
    return Vt.T @ np.diag(1/np.diag(A)) @ S.T @ U.T @ y

In [7]:
w = fit_svd(X_train, y_train, alpha=0.1)

sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
#assert error <= 1e-11
print("prediction error: ", error)

prediction error:  2.1748448821230398e-13


# Everything in a class

In [8]:
class RidgeRegression():
    def __init__(self, fit_intercept=True, method="inverse", alpha=1.0):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.method = method
        self.alpha = alpha
    
    def fit(self, X, y):
        # YOUR CODE HERE
            
        if self.method == 'qr':
            self.w = fit_qr(X, y, self.alpha, self.fit_intercept)
            
        elif self.method == 'svd':
            self.w = fit_svd(X, y, self.alpha, self.fit_intercept)
        
        else:
            self.w = fit_inverse(X, y, self.alpha, self.fit_intercept)
        
    def predict(self, X):
        # YOUR CODE HERE
        
        # with bias
        if self.fit_intercept == True:
            temp = np.ones((len(X), 1))
            X = np.concatenate((temp, X), axis=1)
            
        return (X @ self.w)   

## without bias

In [9]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=False, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=False, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error inverse: ", error)

model_qr = RidgeRegression(fit_intercept=False, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = RidgeRegression(fit_intercept=False, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
assert error_svd <= 1e-11
print("prediction error svd: ", error_svd)

prediction error inverse:  3.093735390874951e-14
prediction error qr:  1.051870032897405e-13
prediction error svd:  8.171669488412137e-14


## with bias

In [10]:
# OTHER APPROACHES
sk_model = Ridge(fit_intercept=True, alpha=0.1)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = RidgeRegression(fit_intercept=True, method="inverse", alpha=0.1)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
assert error <= 1e-11
print("prediction error inverse: ", error)

model_qr = RidgeRegression(fit_intercept=True, method="qr", alpha=0.1)
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
#assert error_qr <= 1e-11
print("prediction error qr: ", error_qr)

model_svd = RidgeRegression(fit_intercept=True, method="svd", alpha=0.1)
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
assert error_svd <= 1e-11
print("prediction error svd: ", error_svd)

prediction error inverse:  1.1875191470499493e-15
prediction error qr:  1.4250229764599388e-15
prediction error svd:  4.600019296831778e-15
